In [41]:
import json
from datetime import datetime
from pathlib import Path
import pprint
import requests

import geocoder


SECRETS_FILE = Path("/home/boggsj/Coding/personal/aipa/config/secrets.json")
with SECRETS_FILE.open('r') as sf:
    SECRETS = json.load(sf)
OPENWEATHER_KEY = SECRETS["openweather_api_key"]

In [27]:
def post_process_weather_data(weather_data):
    def _parse_timestamps(data):
        for k, v in data.items():
            if k in ["dt", "sunrise", "sunset", "moonrise", "moonset"]:
                dt = datetime.fromtimestamp(v)
                data[k] = str(dt)
    if isinstance(weather_data, list):
        for d in weather_data:
            _parse_timestamps(d)
    else:
        _parse_timestamps(weather_data)
    return weather_data
        

In [16]:
geolocation = geocoder.ip("me")
lat, long = geolocation.latlng

r = requests.get(
    "https://api.openweathermap.org/data/3.0/onecall",
    params={
        "lat": lat,
        "lon": long,
        "units": "imperial",
        "exclude": "minutely",
        "appid": OPENWEATHER_KEY
    }
)

r

<Response [200]>

In [33]:
weather_data = r.json()
hourly_weather_data = post_process_weather_data(weather_data["hourly"])
daily_weather_data = post_process_weather_data(weather_data["daily"])
current_weather = post_process_weather_data(weather_data["current"])

In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [45]:
model_name = "HuggingFaceTB/SmolLM3-3B"
device="cuda"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<think>
Okay, let's see. The user wants a weather report based on the JSON data provided. The current weather and the daily forecast are both given. I need to make sure I cover both the current conditions and the expected changes throughout the day.

First, the current weather. The temperature is 60.46°F, with a few clouds (20% coverage). The humidity is 41%, which is pretty high. The feels like temperature is 58.14°F. The wind is coming from 40 degrees, which is a bit to the northeast, and the wind speed is around 9.22 mph. The UV index is 3.68, so sun protection is recommended. Sunrise is at 6:47 AM, sunset at 5:25 PM. The pressure is 1029 hPa, which is normal.

Now the daily forecast. The main points here are the temperature variations. Daytime high is 65.07°F, but the minimum is 42.44°F. The humidity is 39%, which is a bit lower than now. The wind is coming from 28 degrees, slightly different direction. The wind speed is 7.11 mph. The summary says "partly cloudy with clear spells,"

In [ ]:
system_prompt = "You are an expert meteorologist and weather presenter who excels at providing clear and concise weather reports. You are frinedly but professional, and you know how to interpret JSON data."
user_prompt = f"The current weather and today's weather forecast are below in JSON format. Write a 1-2 paragraph weather report for today, including what it's like now and what the day will be like.\n\n*Current weather:*\n{pprint.pformat(current_weather)}\n\n*Daily forecast:*\n{pprint.pformat(daily_weather_data[0])}"

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=2**12, do_sample=True, temperature=0.6, top_p=0.95)

output_ids = generated_ids[0][model_inputs['input_ids'].shape[1]:]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

In [47]:
print(user_prompt)

The current weather and today's weather forecast are below in JSON format. Write a 1-2 paragraph weather report for today, including what it's like now and what the day will be like.

*Current weather:*
{'clouds': 20,
 'dew_point': 36.68,
 'dt': '2025-11-04 11:06:18',
 'feels_like': 58.14,
 'humidity': 41,
 'pressure': 1029,
 'sunrise': '2025-11-04 06:47:29',
 'sunset': '2025-11-04 17:25:58',
 'temp': 60.46,
 'uvi': 3.68,
 'visibility': 10000,
 'weather': [{'description': 'few clouds',
              'icon': '02d',
              'id': 801,
              'main': 'Clouds'}],
 'wind_deg': 40,
 'wind_speed': 9.22}

*Daily forecast:*
{'clouds': 16,
 'dew_point': 35.71,
 'dt': '2025-11-04 12:00:00',
 'feels_like': {'day': 58.39, 'eve': 54.37, 'morn': 38.91, 'night': 47.98},
 'humidity': 39,
 'moon_phase': 0.47,
 'moonrise': '2025-11-04 16:39:00',
 'moonset': '2025-11-04 05:38:00',
 'pop': 0,
 'pressure': 1029,
 'summary': 'Expect a day of partly cloudy with clear spells',
 'sunrise': '2025-11